In [6]:
import requests
from bs4 import BeautifulSoup
import os

# URL of the HTML page containing PDF links
url = 'https://kpmg.com/xx/en/our-insights/eu-tax/etf-557-cjeu-rules-that-the-polish-tax-exemption-limited-to-externally-managed-investment-funds-breaches-eu-law.html'

# Fetch the HTML content
response = requests.get(url)
html_content = response.content

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find all links to PDF files
pdf_links = soup.find_all('a', href=True)
pdf_urls = [link['href'] for link in pdf_links if link['href'].endswith('.pdf')]
# print(pdf_urls)
# Directory to save PDFs
output_dir = 'downloaded_pdfs'
os.makedirs(output_dir, exist_ok=True)

# Download each PDF
for pdf_url in pdf_urls:
    url = "https://assets.kpmg.com/"+ pdf_url
    pdf_response = requests.get(url)
    pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(pdf_response.content)
    print(f'Downloaded: {pdf_filename}')


Downloaded: downloaded_pdfs\etf-557-cjeu-rules-that-the-polish-tax-exemption-limited-to-externally-managed-investment-funds-breaches-eu-law.pdf


In [ ]:
import requests
from bs4 import BeautifulSoup
import os

# URL of the main insights page
main_url = 'https://kpmg.com/xx/en/our-insights.html'

# Fetch the HTML content of the main page
response = requests.get(main_url)
html_content = response.content

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Navigate through the nested structure to find the target section
root = soup.find('body', class_='page basicpage')
container = root.find('div', class_='root container responsivegrid')
cmp_container = container.find('div', id='container-b71de48f42', class_='cmp-container')
main_section = cmp_container.find('div', class_='main container responsivegrid')
# print(main_section)
section = main_section.find('div', class_='section container responsivegrid utils-bleed-bottom')
# print(section)
filterlisting = section.find('div', class_='filterlisting')
# print(filterlisting)
cmp_filterlist = filterlisting.find('div', class_='cmp-filterlist')
# print(cmp_filterlist)
result_container = cmp_filterlist.find('section', class_='cmp-filterlist__result-container')
print(result_container)
res = result_container.find('')
# # Extract all links within the specified section
# article_links = []
# if result_container:
#     links = result_container.find_all('a', class_='cmp-filterlist__tile--action-link', href=True)
#     for link in links:
#         href = link['href']
#         # Ensure the link is absolute
#         if not href.startswith('http'):
#             href = os.path.join('https://kpmg.com', href.lstrip('/'))
#         article_links.append(href)

# # Print or store the collected links
# for article_link in article_links:
#     print(article_link)

# # Optionally, save the links to a file
# with open('collected_links.txt', 'w') as file:
#     for article_link in article_links:
#         file.write(f'{article_link}\n')


<section class="cmp-filterlist__result-container" cmp_filterlist_result_container=""></section>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https://kpmg.com/xx/en/our-insights.html')
wait = WebDriverWait(driver, 10)
article_links = []

try:
    accept_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept All Cookies')]"))
    )
    accept_button.click()
    time.sleep(2)
except Exception as e:
    print("No cookie banner found or already accepted:", e)

def extract_links():
    """Extracts article links from the current page and appends new links."""
    container = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'cmp-filterlist__result-container')))
    links = container.find_elements(By.CLASS_NAME, 'cmp-filterlist__tile--action-link')
    for link in links:
        href = link.get_attribute('href')
        if href not in article_links:
            article_links.append(href)
            if len(article_links) >= 70:
                break

extract_links()

while len(article_links) < 70:
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        pagination_container = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.cmp-filterlist__pagination--numbers'))
        )
        
        driver.execute_script("arguments[0].scrollIntoView();", pagination_container)
        time.sleep(1)
        
        page_buttons = pagination_container.find_elements(By.TAG_NAME, 'button')
        
        active_index = None
        for i, btn in enumerate(page_buttons):
            if 'active' in btn.get_attribute('class'):
                active_index = i
                break
        
        if active_index is not None and active_index < len(page_buttons) - 1:
            next_page_button = page_buttons[active_index + 1]
            next_page_button.click()
            time.sleep(3)
            extract_links()
        else:
            print("No next page available.")
            break
    except Exception as e:
        print("Error during pagination:", e)
        break

article_links = article_links[:70]

print("Total articles collected:", len(article_links))
for link in article_links:
    print(link)

driver.quit()


Total articles collected: 70
https://kpmg.com/xx/en/our-insights/value-creation/a-delicate-balancing-act-between-economic-impact-and-taxation.html
https://kpmg.com/xx/en/our-insights/eu-tax/etf-557-cjeu-rules-that-the-polish-tax-exemption-limited-to-externally-managed-investment-funds-breaches-eu-law.html
https://kpmg.com/xx/en/our-insights/ifrg/2024/ifrs-sme.html
https://kpmg.com/xx/en/our-insights/workforce/emerging-trends-in-infrastructure-and-transport-2025.html
https://kpmg.com/xx/en/our-insights/eu-tax/etf-556-eu-clean-industrial-deal.html
https://kpmg.com/xx/en/our-insights/ifrg/2025/esrs-eu-omnibus.html
https://kpmg.com/xx/en/our-insights/transformation/icgn-japan-conference-2025.html
https://kpmg.com/xx/en/our-insights/regulatory-insights/uk-regulatory-radar-february-2025.html
https://kpmg.com/xx/en/our-insights/eu-tax/etf-555-european-commission-publishes-work-program-for-2025.html
https://kpmg.com/xx/en/our-insights/regulatory-insights/the-growth-and-competitiveness-agenda-f

In [21]:
!pip install webdriver-manager